In [1]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torchsummary import summary
import numpy as np

In [2]:
import random
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

setup_seed(3409)

# 1. Parameter

In [3]:
# dataset
input_shape = 256
num_classes = 2
classes = ('ARMD', 'Normal')

# hyper
batch_size =6
num_epochs = 16
learning_rate = 0.001

# gpu
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Data Pre-processing

In [4]:
# # randomly select 55 images from class 'normal'
# import shutil, random, os
# raw_path = './raw/normal'
# balance_path = './balanced/normal'
# if not os.path.exists(balance_path):
#     os.makedirs(balance_path)

# def balance_files():
#     filenames = random.sample(os.listdir(raw_path), 55)
#     for fname in filenames:
#         srcpath = os.path.join(raw_path, fname)
#         shutil.copy(srcpath, balance_path)
#     shutil.copytree('./raw/armd', './balanced/armd', dirs_exist_ok=True)

# balanced_normal = len([file for file in os.listdir(balance_path)])
# if balanced_normal != 55:
#     balance_files()

In [5]:
# split training images and testing images
import splitfolders
raw = './number'
split = './processed'
splitfolders.ratio(raw, split, seed = 1337, ratio = (0.7, 0.3))

Copying files: 110 files [00:00, 221.12 files/s]


In [6]:
# print(len([file for file in os.listdir('./processed/train/armd')]))
# print(len([file for file in os.listdir('./processed/train/normal')]))
# print(len([file for file in os.listdir('./processed/val/armd')]))
# print(len([file for file in os.listdir('./processed/val/normal')]))

In [7]:
# import images
data_transfrom = transforms.Compose([  
    transforms.ToTensor(),             
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize((input_shape, input_shape)),
])
 
trainset = datasets.ImageFolder('./processed/train', transform = data_transfrom)
testset = datasets.ImageFolder('./processed/val', transform = data_transfrom) 

trainloader = torch.utils.data.DataLoader(dataset = trainset, 
                                          batch_size = batch_size, 
                                          shuffle = True, 
                                          num_workers = 1) 
testloader = torch.utils.data.DataLoader(dataset = testset, 
                                         batch_size = batch_size, 
                                         shuffle = True, 
                                         num_workers = 1)

In [8]:
# images, labels = next(iter(trainloader))

In [9]:
# images.shape

# 3. CNN Model

In [10]:
class CNN(nn.Module):
    def __init__(self, input_shape, in_channels, num_classes):
        super(CNN, self).__init__()
        self.cnn1 = nn.Sequential(nn.Conv2d(in_channels = in_channels, out_channels = 9, 
                                            kernel_size = 5, padding = 2, stride = 1),
                                  nn.BatchNorm2d(9),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.cnn2 = nn.Sequential(nn.Conv2d(in_channels = 9, out_channels = 27, 
                                            kernel_size = 5, padding = 2, stride = 1),
                                  nn.BatchNorm2d(27),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.cnn3 = nn.Sequential(nn.Conv2d(in_channels = 27, out_channels = 81, 
                                            kernel_size = 5, padding = 2, stride = 1),
                                  nn.BatchNorm2d(81),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(81*(input_shape//8)*(input_shape//8), num_classes)

#         self.fc = nn.Linear(27*(input_shape//4)*(input_shape//4), num_classes)
    
    def forward(self, x):
        out = self.cnn1(x)
        out = self.cnn2(out)
        out = self.cnn3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [11]:
model = CNN(input_shape = input_shape, in_channels = 3, num_classes = num_classes)
summary(model, input_size = (3, input_shape, input_shape), batch_size = batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [6, 9, 256, 256]             684
       BatchNorm2d-2           [6, 9, 256, 256]              18
              ReLU-3           [6, 9, 256, 256]               0
         MaxPool2d-4           [6, 9, 128, 128]               0
            Conv2d-5          [6, 27, 128, 128]           6,102
       BatchNorm2d-6          [6, 27, 128, 128]              54
              ReLU-7          [6, 27, 128, 128]               0
         MaxPool2d-8            [6, 27, 64, 64]               0
            Conv2d-9            [6, 81, 64, 64]          54,756
      BatchNorm2d-10            [6, 81, 64, 64]             162
             ReLU-11            [6, 81, 64, 64]               0
        MaxPool2d-12            [6, 81, 32, 32]               0
           Linear-13                     [6, 2]         165,890
Total params: 227,666
Trainable params:

# 4. Model Training

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [13]:
total_batch = len(trainloader)

In [14]:
for epoch in range(num_epochs):
    loss_ave=0
    for batch_idx, (images, labels) in enumerate(trainloader):
#         images = images.to(device)
#         labels = labels.to(device)
        
        out = model(images)
        loss = criterion(out, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_ave += loss.item()
#         if (batch_idx+1) % 4 == 0:
#             print(f'{epoch+1}/{num_epochs}, {batch_idx+1}/{total_batch}: {loss.item():.4f}')
    print(f'{epoch+1}/{num_epochs}, average loss: {loss.item():.4f}')

print('Finished Training')

1/16, average loss: 8.8779
2/16, average loss: 8.3447
3/16, average loss: 7.8120
4/16, average loss: 0.5877
5/16, average loss: 0.1948
6/16, average loss: 0.2687
7/16, average loss: 1.4327
8/16, average loss: 1.8572
9/16, average loss: 0.1463
10/16, average loss: 0.2400
11/16, average loss: 0.1049
12/16, average loss: 0.0083
13/16, average loss: 2.5463
14/16, average loss: 0.8257
15/16, average loss: 0.0965
16/16, average loss: 0.0133
Finished Training


# 5. Model Evaluation

In [15]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
conf_mat = np.zeros((num_classes, num_classes))

# again no gradients needed
with torch.no_grad():
    for images, labels in testloader:
        out = model(images)
        predictions = torch.argmax(out, 1) 
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
            conf_mat[label, prediction] += 1
print(conf_mat)

accuracy = sum(correct_pred.values())/sum(total_pred.values())
print(f'Accuracy: {sum(correct_pred.values())}/{sum(total_pred.values())} = {100*accuracy:.1f} %')
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:3s} is {accuracy:.1f} %')
    
precision = conf_mat[0,0]/(conf_mat[0,0] + conf_mat[1,0])
recall = conf_mat[0,0]/(conf_mat[0,0] + conf_mat[0,1])
print(f'Precision: {precision*100:.1f} %')
print(f'recall: {recall*100:.1f} %')


# print F1-score
f1_score = conf_mat[0,0]/(conf_mat[0,0]+0.5*(conf_mat[0,1]+conf_mat[1,0]))
print(f'F1-score: {f1_score*100:.1f} %')

[[15.  2.]
 [ 0. 17.]]
Accuracy: 32/34 = 94.1 %
Accuracy for class: ARMD is 88.2 %
Accuracy for class: Normal is 100.0 %
Precision: 100.0 %
recall: 88.2 %
F1-score: 93.8 %
